In [ ]:
import cv2
import threading
import os
import sys
from tkinter import Tk, Button

try:
    from moviepy.editor import VideoFileClip, clips_array
except ModuleNotFoundError:
    os.system(f"{sys.executable} -m pip install moviepy")
    from moviepy.editor import VideoFileClip, clips_array

output_dir = os.path.join(os.path.expanduser("~"), "Downloads")
os.makedirs(output_dir, exist_ok=True)

recording = True

def record_webcam(camera_index, output_path):
    cap = cv2.VideoCapture(camera_index)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fps = 30.0
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while recording and cap.isOpened():
        ret, frame = cap.read()
        if ret:
            out.write(frame)
        else:
            break

    cap.release()
    out.release()

def stop_recording():
    global recording
    recording = False
    root.destroy()

def create_montage(video_paths, output_path):
    clips = [VideoFileClip(video) for video in video_paths]
    montage = clips_array([[clip] for clip in clips])
    montage.write_videofile(output_path, codec="libx264")

def main():
    global root
    webcam_indices = [0, 1]  # Adjust this list based on available webcams
    threads = []
    video_paths = []

    for i, index in enumerate(webcam_indices):
        output_path = os.path.join(output_dir, f"webcam_{i}.avi")
        video_paths.append(output_path)
        thread = threading.Thread(target=record_webcam, args=(index, output_path))
        threads.append(thread)
        thread.start()

    root = Tk()
    root.title("Stop Recording")
    stop_button = Button(root, text="Stop", command=stop_recording)
    stop_button.pack(pady=20, padx=20)
    root.mainloop()

    for thread in threads:
        thread.join()

    montage_path = os.path.join(output_dir, "webcam_montage.mp4")
    create_montage(video_paths, montage_path)
    print(f"Montage saved at: {montage_path}")

if __name__ == "__main__":
    main()
